# Introduction

* Lambda 
* Typing
* Monads




In [1]:
%load_ext mypy_ipython

# Lambda

## Syntax

In [2]:
def _identity(x):
  return x

print(_identity)

identity = lambda x: x

print(identity)

identity(1)

<function _identity at 0x7f9c5c31b5e0>
<function <lambda> at 0x7f9c5c31b8b0>


1

## Higher-order functions

Functions that take in functions as arguments or return functions

### Takes in functions as arguments

In [3]:
def _apply(f, x):
  return f(x)

apply = lambda f, x: f(x)

apply(identity, 1)

1

### Returns functions 

In [4]:
def _const(x):
  def func(y):
    return x
  return func

const = lambda x: lambda _: x

const1 = const(1)

const1(0)

1

### Both

In [5]:
def _compose(f, g):
  def func(x):
    return f(g(x))
  return func

compose = lambda f, g: lambda x: f(g(x))

add1 = lambda x: x + 1

compose(add1, add1)(1)

3

## Builtin

### `map`

In [6]:
list(map(add1, [1,2,3]))

[2, 3, 4]

### `filter`

In [7]:
list(filter(lambda x: x < 2, [1,2,3]))

[1]

## `functools`

### partial application

In [8]:
add = lambda x, y: x + y

from functools import partial

add1 = partial(add, 1)

add1(0)

1

### fold

fold is called `reduce` in python.

`reduce(<binary function>, <iterable>, <initializer>)`

`reduce(+, [x1, ..., xn], 0) = 0 + x1 + ... + xn`

In [9]:
from functools import reduce

reduce(add, [1,2,3], 0)

6

## Church encoding
Alegbra data type can be encoded in lambdas. 
  * `bool`
  * natural numbers
  * `list`
  * trees
  * pair
  * disjoint union

### `bool`

In [ ]:
true = lambda x, y: x
false = lambda x, y: y

In [10]:
if_then_else = lambda x, y, z: x(y, z)

_and = lambda x, y: if_then_else(x, y, false)
_or = lambda x, y: if_then_else(x, true, y)
_not = lambda x: if_then_else(x, false, true)
to_bool = lambda x: if_then_else(x, True, False)

print(if_then_else(true, 1, 2))
print(to_bool(_and(false, false)))
print(to_bool(_not(true)))

1
False
False


### natural numbers

In [ ]:
zero = lambda x, f: x
# one = lambda x, f: f(x)
# two = lambda x, f: f(f(x))

In [ ]:
succ = lambda n: lambda x, f: f(n(x, f))

In [11]:
to_int = lambda n: n(0, add1)

one = succ(zero)
two = succ(one)

plus = lambda m, n: m(n, succ)

from functools import partial

times = lambda m, n: m(zero, partial(plus, n))

print(f"[0] = {to_int(zero)}")
print(f"[1] = {to_int(one)}")
print(f"[0 + 2] = {to_int(plus(zero,two))}")
print(f"[1 + 1] = {to_int(plus(one,one))}")
print(f"[1 * 2] = {to_int(times(one,two))}")
print(f"[2 * 2] = {to_int(times(two,two))}")

[0] = 0
[1] = 1
[0 + 2] = 2
[1 + 1] = 2
[1 * 2] = 2
[2 * 2] = 4


# Typing

 * `Any`: any type
 * primitive type: `int`, `bool`, `str`, `float`
 * `Callable`: function types `Callable[[s1, ..., sn], t]`
 * `List`: list types `List[t]`
 * `Tuple`: tuple type `Tuple[t1, ..., tn]`
 * `Dict`: dictionary type `Dict[s, t]`
 * `Optional`: optional type `Optional[T]`
 * `Union`: union type `Union[t1, ..., tn]`
 " `Literal`: literal types `Literal[l1, ..., ln]`
 * `Type`: type of types `Type[t]`
 
 Parametric Polymorphism (Genericity)

 * `TypeVar`: type variable
 

## Syntax

In [12]:
from typing import Callable, List, TypeVar, Tuple, Dict, Optional, Generic
from dataclasses import dataclass
from abc import ABC
from __future__ import annotations



In [13]:
def _add1(x: int) -> int:
  return x + 1

# add1('a') type error


In [14]:
add1: Callable[[int], int] = lambda x: x + 1

# add1('a') type error

## Typing functions

In [15]:
# define type variables
R = TypeVar("R")
S = TypeVar("S")
T = TypeVar("T")

# apply: Callable[[Callable[[S], T], S], T]

# const: Callable[[S], Callable[[T], S]]

# compose: Callable[[Callable[[S], T], Callable[[R], S]], Callable[[R], T]]

# map: Callable[[Callable[[S], T], List[S]], List[T]]

# filter: Callable[[Callable[[T], bool], List[T]], List[T]]

# reduce: Callable[[Callable[[R, S], T], List[S], R], T]

## Custom generic types

Inheriting the `Generic` type allows defining generic types `Generic[s1, ..., sn]`

In [ ]:
class Tree(Generic[T]):
  def __init__(self, value: T, subtrees: List[Tree[T]]) -> None:
    self.value = value
    self.subtrees = subtrees

In [16]:
a: Tree[int] = Tree(1, [Tree(2, []), Tree(2, [])])

def size(a: Tree[int]) -> int:
  return 1 + sum(map(size, a.subtrees))

size(a)

3

# Monads

Monadic programming provides two generic operator that are useful for implementing a wide range of effectful operators such as:

 * Null value
 * Error handling
 * I/O
 * State
 * Choice
 * Backtracking
 * Asynchronicity

And allows composing these effects via constructs such as monad transformers.

`tx-functional`

Other libraries:
 * `PyMonad`
 * `fn.py`
 * `OSlash`

## Operators

Given a monad `m`

```
pure: Callable[[T], m[T]]
bind: Callable[[m[S], Callable[[S], m[T]]], m[T]]
```

## Example: Maybe monad

In [ ]:
class Maybe(Generic[S]):
    
    def bind(self, f: Callable[[S], Maybe[T]]) -> Maybe[T]:
        if isinstance(self, Nothing):
            return Nothing()
        elif isinstance(self, Just):
            return f(self.value)
        else:
            raise RuntimeError(f"unsupported type of {self}")

            
def pure(a: T) -> Maybe[T]:
    return Just(a)


class Just(Maybe[T]):
    def __init__(self, value: T) -> None:
        self.value = value

        
class Nothing(Maybe[T]):
    pass

In [17]:
import requests

In [18]:
def get(url: str) -> Maybe[str]:
    resp = requests.get(url)
    if resp.status_code != 200:
        return Nothing()
    else:
        return Just(resp.text)

In [19]:
def post(url: str) -> Callable[[str], Maybe[str]]:
    def func(data):
        resp = requests.post(url, data=data)
        if resp.status_code != 200:
            return Nothing()
        else:
            return Just(resp.text)
    return func    

In [20]:
url1 = "https://example.com/"
url2 = "https://example.com/non_existent_path"

In [21]:
get(url1)

In [22]:
get(url2)

In [23]:
get(url1).bind(post(url2))

In [24]:
get(url1).bind(post(url1))